### Reinitialize the file

In [ ]:
# warning !! remove all except this script
## remove -i argument to run
!rm `(ls | grep -v '.ipynb')`

In [ ]:
pdbid='2jpz'
drug='EWV'
jobname = f'{pdbid}_{drug}' if not drug is None else pdbid

In [ ]:
import os
ff_folder  = os.path.realpath("../../../smsl/opt/force_field")
mdp_folder = os.path.realpath("../../../smsl/opt/mdp")

### G-quadruplex Prepare

In [ ]:
!cp ../../c.gromacs.run_md/1.create_system/*{drug}* ./
!cp ../../c.gromacs.run_md/1.create_system/*.itp ./
!cp ../../c.gromacs.run_md/1.create_system/*.top ./
!cp ../../c.gromacs.run_md/1.create_system/{pdbid}.gro.pdb ./
!cp ../5.Output/{jobname}.gro.pdb ./

In [ ]:
# 畫完box後座標會平移
# 因此最後要將座標再align回原本座標
# 因為不建立水箱 因此 -d 數值隨意(但不要太小以防止因為pbc而導致結構被切一半) 
!gmx editconf -f {jobname}.gro.pdb -o {jobname}_dode.pdb -c yes -d 1.5 -bt dodecahedron

In [ ]:
cmd = f'''\
r EWV | r DG
name 5 {drug}_DG
q
'''
!echo -e "{cmd}" | gmx make_ndx -f {jobname}.gro.pdb -o {jobname}.ndx

### 因為藥物跟loop空間上會重疊，因此將藥物往下位移一點

In [ ]:
cmd=f'''\
pymol >>
load {jobname}_dode.pdb
extract {drug}, resn {drug}
show_as sticks
color green  , resn DA and name *C*
color red    , resn DT and name *C*
color magenta, resn DG and name *C*
color cyan  , resn POH and name *C*
color wheat , resn EWV and name *C*
sele {drug}
[click by mouse] (sele) > action > drag_coordinate > 
    shift+middle mouse button to [translate drug]
    shift+right  mouse button to [rotate drug] > Done
set retain_order, 1
save {jobname}_dode.1.pdb, enable
>> EOF
'''
print(cmd)

## First Cycle of EM1

In [ ]:
## 固定藥物和核心區域(DG)
freeze_cmd = f'''\
freezegrps      = {drug}_DG
freezedim       = Y Y Y
'''

In [ ]:
!echo "{freeze_cmd}"        > min1.mdp
!cat {mdp_folder}/min1.mdp >> min1.mdp

In [ ]:
!gmx grompp -f min1.mdp -c {jobname}_dode.1.pdb -r {jobname}_dode.1.pdb -p topol.top -o min1.tpr -n {jobname}.ndx -maxwarn 1
!gmx mdrun -deffnm min1 -c min1.pdb -v

### First cycle of EM2

In [ ]:
!echo "{freeze_cmd}"        > min2.mdp
!cat {mdp_folder}/min1.mdp >> min2.mdp

In [ ]:
!gmx grompp -f min2.mdp -c min1.pdb -r {jobname}_dode.1.pdb -p topol.top -o min2.tpr -n {jobname}.ndx -maxwarn 1
!gmx mdrun -deffnm min2 -c min2.pdb -v

### Second cycle
### 現在上層loop往上移一點了，因此我們可以再將藥物往上移一點

In [ ]:
cmd=f'''\
pymol >>
load min2.pdb
extract {drug}, resn {drug}
show_as sticks
color green  , resn DA and name *C*
color red    , resn DT and name *C*
color magenta, resn DG and name *C*
color cyan  , resn POH and name *C*
color wheat , resn EWV and name *C*
sele {drug}
[click by mouse] (sele) > action > drag_coordinate > 
    shift+middle mouse button to [translate drug]
    shift+right  mouse button to [rotate drug] > Done
set retain_order, 1
save {jobname}_dode.2.pdb, enable
>> EOF
'''
print(cmd)

### Second cycle of EM1

In [ ]:
!gmx grompp -f min1.mdp -c {jobname}_dode.2.pdb -r {jobname}_dode.2.pdb -p topol.top -o min1.tpr -n {jobname}.ndx -maxwarn 1
!gmx mdrun -deffnm min1 -c min1.pdb -v

### Second cycle of EM2

In [ ]:
!gmx grompp -f min2.mdp -c min1.pdb -r {jobname}_dode.2.pdb -p topol.top -o min2.tpr -n {jobname}.ndx -maxwarn 1
!gmx mdrun -deffnm min2 -c min2.pdb -v

### Third cycle
### 現在將藥物換成原本的pose再做最後一次能量最小化

In [ ]:
cmd=f'''\
pymol >>
load min2.pdb
extract {drug}_min2, min2 and resn {drug}
load {jobname}_dode.pdb
extract {drug}, {jobname} and resn {drug}
show_as sticks
color green  , resn DA and name *C*
color red    , resn DT and name *C*
color magenta, resn DG and name *C*
color cyan  , resn POH and name *C*
color wheat , resn EWV and name *C*
set retain_order, 1
save {jobname}_dode.3.pdb, min2 or {drug}
>> EOF
'''
print(cmd)

### Third cycle of EM1

In [ ]:
!gmx grompp -f min1.mdp -c {jobname}_dode.3.pdb -r {jobname}_dode.3.pdb -p topol.top -o min1.tpr -n {jobname}.ndx -maxwarn 1
!gmx mdrun -deffnm min1 -c min1.pdb -v

### Second cycle of EM2

In [ ]:
!gmx grompp -f min2.mdp -c min1.pdb -r {jobname}_dode.2.pdb -p topol.top -o min2.tpr -n {jobname}.ndx -maxwarn 1
!gmx mdrun -deffnm min2 -c min2.pdb -v

In [ ]:
cmd=f'''\
pymol >>
load min2.pdb
load {jobname}.gro.pdb
align min2, {jobname}.gro
set retain_order, 1
save {jobname}.final.gro.pdb, min2
>> EOF
'''
print(cmd)

最終 {jobname}.final.gro.pdb 就是分子模擬的初始結構